# [LAB 05] Data Preprocessing 3 / Feature Engineering

## 5. 데이터 스케일링

In [ ]:
#----------------------강사님 설명-------------------------------
# 스케일링 when?: 변수당 측정단위 밑 범위 차이가 심할 때

# if 이상치 존재 확인하면:
    # 없는거 확인하면: MinMaxScaler, MaxAbsScaler 
    # 있는거 확인하면: StandardScaler, RobustScaler

# Scaler 마다 있는 계산식 외우는거 귀찮으니까 
# 걍 scikit learn 쓰는걸로!
    # 주의: scikit learn은 2차원 데이터프레임화 한 
    #       변수형태로 가져와야됨!

# 표본집단의 표준편차: ㄴs
# 모집단의 표준편차: σ
# 모집단의 평균: μ
# 표본집단의 평균: X바

# 포토폴리오에는 파이썬 코드 절대 넣는거 아니다!!
    # 대신, '계산은 아래와 같은 계산식을 이용하였다: ~' 같이
    # 서술형식으로 적어야 됨

# 중앙값(x물결바)가 식에 있다면 '무조건! 
# 해당 식은 이상치의 영향을 받지 않겠구나' 라는 생각이 
# 자연스럽게 되야 식의 의미를 진짜 이해할 수 있는거임!
#------------------------------------------------------

In [ ]:
# 스케일링 적용 기준:
    # 1-1. 적용 안하는 case:
        # 더미변수 처리
        # 머신러닝: 트리기반 모델
    # 1-2. 적용하는 case: 
        # 머신러닝: 거리기반(KNN, SVM), 경사하강법 기반(~NN, Linear Regression)
    # 2-1. 이상치가 없을 때:
        # inMaxScaler, MaxAbsScaler
    # 2-2. 이상치가 있을 때:
        # StandardScaler, RobustScaler

### #01. 준비작업

In [ ]:
# 1. 라이브러리 참조:
from hossam import load_data
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, RobustScaler

In [6]:
# 2. 샘플데이터 가져오기:
origin = load_data('nursing_grades')
origin

[data] https://data.hossam.kr/data/lab05/nursing_grades.xlsx
[desc] 어느 간호학과 대학원에 지원한 학생들에 대한 합격/불합격 여부를 조사한 가상의 데이터(메타데이터 없음)
[!] Cannot read metadata


,이름,성별,필기점수,학부성적,병원경력,합격여부
접수코드,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격
NRS0002,최지호,남,660,3.67,3,합격
NRS0003,김하준,남,800,4.00,1,합격
NRS0004,임아윤,여,640,3.19,4,합격
NRS0005,강하준,남,520,2.93,4,불합격
...,...,...,...,...,...,...
NRS0396,박지유,여,620,4.00,2,불합격
NRS0397,조하은,여,560,3.04,3,불합격
NRS0398,박하윤,여,460,2.63,2,불합격


### #02. Min-Max Scaler (정규화)

In [ ]:
# 1. 방법1. 직접 계산하기 (비추):
#-----------------계산식-------------------
# X_scaled = (X - X_min) / (X_max - X_min)
#------------------------------------------
# - 모든 데이터의 범위를 '0 ~ 1'로 변환하는 방법
# -> 데이터에서 최솟값을 0, 최댓값을 1로 매핑
# -> 강점: 데이터 분포를 유지하면서 값을 특정 범위로 축소시킴
minmax_df = origin.copy()

x = minmax_df['필기점수']
xmin = x.min()
xmax = x.max()

minmax_df['필기점수_MinMax(1)'] = (x - xmin) / (xmax - xmin)
minmax_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,필기점수_MinMax(1)
접수코드,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,0.275862
NRS0002,최지호,남,660,3.67,3,합격,0.758621
NRS0003,김하준,남,800,4.00,1,합격,1.000000
NRS0004,임아윤,여,640,3.19,4,합격,0.724138
NRS0005,강하준,남,520,2.93,4,불합격,0.517241


In [10]:
# 2. 방법2. Sklearn 활용하기 (강추)

# step 1) 특정 필드만 추출:
x = minmax_df[['필기점수']]
x.head()

,필기점수
접수코드,
NRS0001,380
NRS0002,660
NRS0003,800
NRS0004,640
NRS0005,520


In [11]:
# step 2) 스케일링 적용: .MinMaxScaler()
scaler = MinMaxScaler()
minmax_df['필기점수_MinMax(2)'] = scaler.fit_transform(x)
minmax_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,필기점수_MinMax(1),필기점수_MinMax(2)
접수코드,,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,0.275862,0.275862
NRS0002,최지호,남,660,3.67,3,합격,0.758621,0.758621
NRS0003,김하준,남,800,4.00,1,합격,1.000000,1.000000
NRS0004,임아윤,여,640,3.19,4,합격,0.724138,0.724138
NRS0005,강하준,남,520,2.93,4,불합격,0.517241,0.517241


### #03. Max-Abs Scaler

In [ ]:
# 1. 방법1. 직접 계산하기:
#-----------------계산식-------------------
# X_scaled = X / max(|X|)
#------------------------------------------
# - 절댓값 기준으로 '-1 ~ 1' 범위로 스케일링
# -> 음수/양수 모두 유지, 희소행렬(sparse matrix)에 유리
abs_df = origin.copy()
x = abs_df['필기점수']
abs_df['필기점수_MaxAbs(1)'] = x / x.abs().max()
abs_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,필기점수_MaxAbs(1)
접수코드,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,0.475
NRS0002,최지호,남,660,3.67,3,합격,0.825
NRS0003,김하준,남,800,4.00,1,합격,1.000
NRS0004,임아윤,여,640,3.19,4,합격,0.800
NRS0005,강하준,남,520,2.93,4,불합격,0.650


In [16]:
# 2. 방법2. Sklearn 활용하기:
scaler = MaxAbsScaler()
x = abs_df[['필기점수']]
abs_df['필기점수_MaxAbs(2)'] = scaler.fit_transform(x)
abs_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,필기점수_MaxAbs(1),필기점수_MaxAbs(2)
접수코드,,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,0.475,0.475
NRS0002,최지호,남,660,3.67,3,합격,0.825,0.825
NRS0003,김하준,남,800,4.00,1,합격,1.000,1.000
NRS0004,임아윤,여,640,3.19,4,합격,0.800,0.800
NRS0005,강하준,남,520,2.93,4,불합격,0.650,0.650


### #04. StandardScaler (z-score, 표준화)

In [20]:
# 1. 방법1. 직접 계산하기:
#-----------------계산식-------------------
# X_scaled = (X - Xbar) / s x X
# (Xbar = 표본평균, s = 표본표준편차)
#------------------------------------------
# - 데이터를 평균: 0, 표준편차: 1인 표준정규분포를 따르도록 변환하는 식
# -> 강점: 데이터를 정규분포로 근사시켜서 이상치에 덜 민감하게 만들어줌
std_df = origin.copy()
x = std_df['학부성적']
std_df['학부성적_Standard(1)'] = (x - x.mean()) / x.std()
std_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,학부성적_Standard(1)
접수코드,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,0.578348
NRS0002,최지호,남,660,3.67,3,합격,0.736008
NRS0003,김하준,남,800,4.00,1,합격,1.603135
NRS0004,임아윤,여,640,3.19,4,합격,-0.525269
NRS0005,강하준,남,520,2.93,4,불합격,-1.208461


In [18]:
# 2. 방법2. Sklearn 활용하기:
scaler = StandardScaler()
x = std_df[['학부성적']]
std_df['학부성적_Standard(2)'] = scaler.fit_transform(x)
std_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,학부성적_Standard(1),학부성적_Standard(2)
접수코드,,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,0.578348,0.579072
NRS0002,최지호,남,660,3.67,3,합격,0.736008,0.736929
NRS0003,김하준,남,800,4.00,1,합격,1.603135,1.605143
NRS0004,임아윤,여,640,3.19,4,합격,-0.525269,-0.525927
NRS0005,강하준,남,520,2.93,4,불합격,-1.208461,-1.209974


### #04. Robust Scaler

In [21]:
# 1. 방법1. 직접 계산하기:
#-----------------계산식-------------------
# X_scaled = (X - X~bar) / iqr
# (X~bar = 중앙값, iqr: 사분위 간 범위)
#------------------------------------------
# - 이상치가 존재할 경우 이상치의 영향을 최소화하여 데이터를 스케일링 하는 방법
# -> 이상치가 포함된 데이터를 표준화 또는 정규화할 때, 
#    이상치의 영향으로 전체 데이터의 분포가 왜곡됨
# -> 강점: RobustScaler는 이 문제를 해결하기 위해 
#         중앙값과 사분위수를 사용해서 데이터 스케일링 함
# -> 잘 안씀!!
rb_df = origin.copy()
x = rb_df['병원경력']
m = x.median()
iqr = x.quantile(0.75) - x.quantile(0.25)

rb_df['병원경력_Robust(1)'] = (x - m) / iqr
rb_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,병원경력_Robust(1)
접수코드,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,1.0
NRS0002,최지호,남,660,3.67,3,합격,1.0
NRS0003,김하준,남,800,4.00,1,합격,-1.0
NRS0004,임아윤,여,640,3.19,4,합격,2.0
NRS0005,강하준,남,520,2.93,4,불합격,2.0


In [22]:
# 2. 방법2. Sklearn 활용하기:
scaler = RobustScaler()
x = rb_df[['병원경력']]
rb_df['병원경력_Standard(2)'] = scaler.fit_transform(x)
rb_df.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,병원경력_Robust(1),병원경력_Standard(2)
접수코드,,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,1.0,1.0
NRS0002,최지호,남,660,3.67,3,합격,1.0,1.0
NRS0003,김하준,남,800,4.00,1,합격,-1.0,-1.0
NRS0004,임아윤,여,640,3.19,4,합격,2.0,2.0
NRS0005,강하준,남,520,2.93,4,불합격,2.0,2.0
